# SPARQL query

In [ ]:
import pandas as pd
import numpy as np
import haversine as hs
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON, CSV

### Input parameters

In [ ]:
#latitude
#longitude
#radius

#### Example setting

In [ ]:
# endpoint
endpoint = 'https://query.wikidata.org/sparql'

# query variables
latitude  = '49.487777777' # Mannheim
longitude = '8.466111111'  # Mannheim

#latitude  = '49.410833333333'  # Heidelberg
#longitude = '8.7063888888889'  # Heidelberg


#obj_class = '' 
radius    = '15'
limit     = '30'

### Query

Description of the retrieved variables in the sparql query
- ?x : Resourcee of the thing (a specific street, chruch, etc.) we search
    - ?xLabel : Label of that resource
    - ?otherLocation : Coordinates of the respective resource
- ?somebody : identifier of the person the resource is named after
    - ?somebodyLabel : Label of that person
- ?classLabel : What kind of thing are we looking at (church, street, venue...)
- ?dist : distance between current location and the location of the retrieved resource



In [ ]:

# Query for the table
query = '''
        prefix bd:       <http://www.bigdata.com/rdf#>
        prefix geo:      <http://www.opengis.net/ont/geosparql#>
        prefix wd:       <http://www.wikidata.org/entity/>
        prefix wdt:      <http://www.wikidata.org/prop/direct/>
        prefix wikibase: <http://wikiba.se/ontology#>


        SELECT ?xLabel ?x ?somebodyLabel ?somebody ?otherLocation 
               (GROUP_CONCAT(?classLabel; separator=', ') AS ?classdescription)
        WHERE
        {{
          BIND('Point({longitude} {latitude})'^^geo:wktLiteral AS ?currentLocation).
          SERVICE wikibase:around {{
              ?x wdt:P625 ?otherLocation. 
              bd:serviceParam wikibase:center ?currentLocation. 
              bd:serviceParam wikibase:radius '{radius}'. 
          }}
          # x is named after
          ?x wdt:P138 ?somebody .
          
          # that somebody is a human
          ?somebody wdt:P31 wd:Q5 .

          # which Class is x (e.g. Street, church)
          ?x wdt:P31  ?class . 
          
          # the class has a german label 
          ?class rdfs:label ?classLabel .
          FILTER( lang(?classLabel) = "de" )
          
          # Retrieve Labels
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "de". }} 
          
        }} 
        GROUP BY ?xLabel ?x ?somebodyLabel ?somebody ?otherLocation
        LIMIT {limit}
        
        '''.format(longitude = longitude, latitude = latitude, radius = radius, limit = limit)
print(query)

In [ ]:

# Query for the map
# mapQuery = '''
#<iframe style="width: 80vw; height: 50vh; border: none;" 
#src="https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%20%20%20%20%20%20%20%20
#prefix%20bd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23%3E%0A%20%20%20%20%20%20%20%20
#prefix%20geo%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.opengis.net%2Font%2Fgeosparql%23%3E%0A%20%20%20%20%20%20%20%20
#prefix%20wd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0A%20%20%20%20%20%20%20%20
#prefix%20wdt%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0A%20%20%20%20%20%20%20%20
#prefix%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0A%0A%0A%20%20%20%20%20%20%20%20
#SELECT%20%3FxLabel%20%3Fx%20%3FsomebodyLabel%20%3Fsomebody%20%3FclassLabel%20%3FotherLocation%20%3Fdist%0A%20%20%20%20%20%20%20%20
#WHERE%0A%20%20%20%20%20%20%20%20%7B%0A%20%20%20%20%20%20%20%20%20%20
#BIND('Point({longitude}%20{latitude})'%5E%5Egeo%3AwktLiteral%20AS%20%3FcurrentLocation).%0A%20%20%20%20%20%20%20%20%20%20
#SERVICE%20wikibase%3Aaround%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP625%20%3FotherLocation.%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20
#bd%3AserviceParam%20wikibase%3Acenter%20%3FcurrentLocation.%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20
#bd%3AserviceParam%20wikibase%3Aradius%20'{radius}'.%20%0A%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%23%20
#x%20is%20named%20after%0A%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP138%20%3Fsomebody%20.%0A%0A%20%20%20%20%20%20%20%20%20%20%23%20
#which%20Class%20is%20x%20(e.g.%20Street%2C%20church)%0A%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP31%20%20%3Fclass%20.%20%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20
#x%20is%20a%20Street%0A%20%20%20%20%20%20%20%20%20%20%23%3Fx%20wdt%3AP31%20wd%3AQ79007%20.%0A%0A%20%20%20%20%20%20%20%20%20%20%23%20
#hat%20Staat%20Deutschland%0A%20%20%20%20%20%20%20%20%20%20%23%20%3Fx%20wdt%3AP17%20%20%3FQ183%20.%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20Retrieve%20Labels%0A%20%20%20%20%20%20%20%20%20%20
#SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22de%22.%20%7D%20%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20
#Compute%20Distance%0A%20%20%20%20%20%20%20%20%20%20%23%20
#BIND(geof%3Adistance(%3FotherLocation%2C%20%3FcurrentLocation)%20as%20%3Fdist)%20%0A%20%20%20%20%20%20%20%20%7D%20%0A%0A%20%20%20%20%20%20%20%20
#LIMIT%20{limit}%0A%20%20%20%20%20%20%20%20%23ORDER%20BY%20%3Fdist" 
#referrerpolicy="origin" sandbox="allow-scripts allow-same-origin allow-popups"></iframe>'''.format(longitude = longitude, latitude = latitude, radius = radius, limit = limit)

mapQuery = '''
<iframe style="width: 80vw; height: 50vh; border: none;" src="https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%20%20%20%20%20%20%20%20prefix%20bd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23%3E%0A%20%20%20%20%20%20%20%20prefix%20geo%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.opengis.net%2Font%2Fgeosparql%23%3E%0A%20%20%20%20%20%20%20%20prefix%20wd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0A%20%20%20%20%20%20%20%20prefix%20wdt%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0A%20%20%20%20%20%20%20%20prefix%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0A%0A%0A%20%20%20%20%20%20%20%20SELECT%20%3FxLabel%20%3Fx%20%3FsomebodyLabel%20%3Fsomebody%20%3FotherLocation%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20(GROUP_CONCAT(%3FclassLabel%3B%20separator%3D'%2C%20')%20AS%20%3Fclassdescription)%0A%20%20%20%20%20%20%20%20WHERE%0A%20%20%20%20%20%20%20%20%7B%0A%20%20%20%20%20%20%20%20%20%20BIND('Point({longitude}%20{latitude})'%5E%5Egeo%3AwktLiteral%20AS%20%3FcurrentLocation).%0A%20%20%20%20%20%20%20%20%20%20SERVICE%20wikibase%3Aaround%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP625%20%3FotherLocation.%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20bd%3AserviceParam%20wikibase%3Acenter%20%3FcurrentLocation.%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20bd%3AserviceParam%20wikibase%3Aradius%20'{radius}'.%20%0A%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%23%20x%20is%20named%20after%0A%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP138%20%3Fsomebody%20.%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20that%20somebody%20is%20a%20human%0A%20%20%20%20%20%20%20%20%20%20%3Fsomebody%20wdt%3AP31%20wd%3AQ5%20.%0A%0A%20%20%20%20%20%20%20%20%20%20%23%20which%20Class%20is%20x%20(e.g.%20Street%2C%20church)%0A%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP31%20%20%3Fclass%20.%20%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20the%20class%20has%20a%20german%20label%20%0A%20%20%20%20%20%20%20%20%20%20%3Fclass%20rdfs%3Alabel%20%3FclassLabel%20.%0A%20%20%20%20%20%20%20%20%20%20FILTER(%20lang(%3FclassLabel)%20%3D%20%22de%22%20)%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20Retrieve%20Labels%0A%20%20%20%20%20%20%20%20%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22de%22.%20%7D%20%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%7D%20%0A%20%20%20%20%20%20%20%20GROUP%20BY%20%3FxLabel%20%3Fx%20%3FsomebodyLabel%20%3Fsomebody%20%3FotherLocation%0A%20%20%20%20%20%20%20%20LIMIT%20{limit}%0A%20%20%20%20%20%20%20%20" referrerpolicy="origin" sandbox="allow-scripts allow-same-origin allow-popups"></iframe>
'''.format(longitude = longitude, latitude = latitude, radius = radius, limit = limit)

mapQuery

In [ ]:
# Initializing SPARQL Wrapper and querying
sparql = SPARQLWrapper(endpoint)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


In [ ]:
# Initializing function to compute ditance
def cumputeDistance(newpoint, currentLongitude, currentLatitude):
    newLocation     = np.array([float(x) for x in newpoint[6:-1].split(' ')])
    currentLocation = np.array([float(longitude), float(latitude)])
    distance = hs.haversine(newLocation,currentLocation)
    return distance

# Results
df = pd.DataFrame(results['results']['bindings'])
for col in df.columns:
    df[col] = df[col].apply(lambda x: x['value'])

# compute ditance    
df['distance'] = df['otherLocation'].apply(lambda point: cumputeDistance(point, longitude, latitude))

# Sort according to distance
df = df.sort_values(by=['distance'])

df.head(10)



In [ ]:
## Test computing distance between two points

# Coordinates of Christuskirche  Q1087367
p1 = 'Point(8.4807 49.4846)'

# Coordinates of St-Pius-Kirche Q1405432
p2 = 'Point(8.51166 49.4798)'

loc1 = np.array([float(x) for x in p1[6:-1].split(' ')])
loc2 = np.array([float(x) for x in p2[6:-1].split(' ')])

# Calculating distance using haversine package | km per default
distance = hs.haversine(loc1,loc2)  
print('Calculated distance:        ', distance)
print('Sparql distance in Wikidata: 2.299291884545297')

# Link to control query in wikidata
# https://query.wikidata.org/#prefix%20unit%3A%20%3Chttp%3A%2F%2Fqudt.org%2Fvocab%2Funit%23%3E%0A%0ASELECT%20%3Fdist%0AWHERE%20%7B%0A%20%20%23%20Berlin%20coordinates%0A%20%20wd%3AQ1405432%20wdt%3AP625%20%3Floc1.%20%0A%20%20wd%3AQ1087367%20wdt%3AP625%20%3Floc2%20.%0A%20%20%0A%20%20BIND%28geof%3Adistance%28%3Floc1%2C%20%3Floc2%2C%20unit%3AKilometer%29%20as%20%3Fdist%29%20%0A%7D%20%0AORDER%20BY%20%3Fdist